In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

# Read scenario data

Refer to **read_data.ipynb** for insight regarding how the data is generated and structured.

In [6]:
with open('Data/250_scenarios.json') as f:
    scenarios = json.load(f)

scenarios.keys()

dict_keys(['Sample Scenarios', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '

# One-price Scheme

*(Task 1.a)*

## Run model

In [47]:
OMEGA = len(scenarios.keys()) - 1 # number of scenarios
T = 24 # number of hours
PI = 1 / OMEGA # probability of each scenario - assumed to be equal
WIND_CAPACITY = 200 #MWh

direction = gb.GRB.MAXIMIZE #Min / Max

m = gb.Model() # Create a Gurobi model  

#============= Variables =============
p_DA = m.addVars(T, lb=0, ub=gb.GRB.INFINITY, name="p_DA") # day-ahead power bid
delta = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="delta") # decision variable for the power imbalance - can be negative
price_coeff = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="K") # price coefficient for the imbalance price wrt. the day-ahead price

#============= Objective function =============
# Set objective function - note that the day-ahead price is factored out of the sum
obj = gb.quicksum(PI * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * (p_DA[t] + price_coeff[t,w] * delta[t,w]) for t in range(T) for w in range(OMEGA))
m.setObjective(obj, direction)

#============= Day-ahead power bid limits ============

#Upper limit is the nominal wind power
m.addConstrs(p_DA[t] <= WIND_CAPACITY for t in range(T))

#============= Power imbalance definition (realized - bid) ===============
m.addConstrs(delta[t,w] == scenarios[str(w)]['Wind Power [MW]'][t] - p_DA[t] for t in range(T) for w in range(OMEGA))

#============= Price coefficient definition ===============
# the system balance parameter is 0 if the system has a surplus and 1 if it has a deficit
m.addConstrs(price_coeff[t,w] == 1.2 * scenarios[str(w)]['System Balance State'][t] + 0.9 * (1 - scenarios[str(w)]['System Balance State'][t]) for t in range(T) for w in range(OMEGA))

#============= Display and run model =============
m.update()
#m.display()
m.optimize()

#============= Results =============
if m.status == gb.GRB.OPTIMAL:
    #initialization
    results = {}
    for scenario in range(OMEGA):
        df = pd.DataFrame(columns=['Hour', 'DA Price [EUR/MWh]', 'Wind Power [MW]', 'DA Bid [MW]', 'Imbalance [MW]', 'DA Profit [EUR]', 'Balancing Profit [EUR]', 'System State', 'Balancing Price Coefficient'])
        
        for t in range(T):
            df.loc[t] = [t, 
                         scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t], 
                         scenarios[str(scenario)]['Wind Power [MW]'][t], p_DA[t].x, 
                         delta[t,scenario].x, scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * p_DA[t].x, 
                         price_coeff[t,scenario].x * scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * delta[t,scenario].x, 
                         scenarios[str(scenario)]['System Balance State'][t], price_coeff[t,scenario].x]
        df['Total Profit [EUR]'] = df['DA Profit [EUR]'] + df['Balancing Profit [EUR]']

        df['Hour'] = df['Hour'].astype(int)
        df['System State'] = df['System State'].astype(int)
        df['System State'] = df['System State'].apply(lambda x: 'Deficit' if x == 1 else 'Surplus')
        df.set_index('Hour', inplace=True)
        results[scenario] = df.copy(deep=True)

    print('-----------------------------------------------')
    print('Objective value (expected profit): %.2f EUR' % m.objVal)
    print('-----------------------------------------------')
    print('Day-ahead bids:')
    average_hourly_profit = np.mean([results[w]['Total Profit [EUR]'] for w in range(OMEGA)], axis=0)

    for t in range(T):
        print('Hour %d | Bid: %.2f MW | Average Profit: %.2f EUR' % (t, p_DA[t].x, average_hourly_profit[t]))

    print('Sum of average profits: %.2f EUR' % np.sum(average_hourly_profit))
    print('-----------------------------------------------')
    print('Runtime: %f ms' % (m.Runtime * 1e3))
else:
    print("Optimization was not successful.")     

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12024 rows, 12024 columns and 18024 nonzeros
Model fingerprint: 0x0df022a9
Model has 5982 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+02]
  QObjective range [8e-05, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 2e+02]
Presolve removed 12024 rows and 12024 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.01 work units)
Optimal objective 1.89356493e+05
-----------------------------------------------
Objective value (expected profit): 189356.49 EUR
-----------------------------------------------
Day-ahead bids:
Hour 0 | Bid: 0.00 MW | Average Profit: 6038.56 EUR
Hour 1 | Bid: 0.00

In [42]:
results[3]

,DA Price [EUR/MWh],Wind Power [MW],DA Bid [MW],Imbalance [MW],DA Profit [EUR],Balancing Profit [EUR],System State,Balancing Price Coefficient,Total Profit [EUR]
Hour,,,,,,,,,
0,52.900002,119.879967,0.0,119.879967,0.0000,7609.980562,Deficit,1.2,7609.980562
1,51.160000,129.306142,0.0,129.306142,0.0000,7938.362667,Deficit,1.2,7938.362667
2,49.810001,138.239461,0.0,138.239461,0.0000,8262.849222,Deficit,1.2,8262.849222
3,49.029999,147.554672,0.0,147.554672,0.0000,8681.526484,Deficit,1.2,8681.526484
4,52.470001,153.003434,0.0,153.003434,0.0000,9633.708373,Deficit,1.2,9633.708373
5,58.970001,149.965008,0.0,149.965008,0.0000,7959.092989,Surplus,0.9,7959.092989
6,62.410000,135.060010,200.0,-64.939990,12482.0000,-4863.485767,Deficit,1.2,7618.514233
7,66.550003,135.064239,0.0,135.064239,0.0000,8089.672936,Surplus,0.9,8089.672936
8,69.910004,128.959107,200.0,-71.040893,13982.0008,-4469.822175,Surplus,0.9,9512.178625


## Visualize results

In [ ]:
...

# Two-price Scheme

*(Task 1.b)*

## Run model

In [ ]:
OMEGA = len(scenarios.keys()) - 1 # number of scenarios
T = 24 # number of hours
PI = 1 / OMEGA # probability of each scenario - assumed to be equal
WIND_CAPACITY = 200 #MWh

direction = gb.GRB.MAXIMIZE #Min / Max

m = gb.Model() # Create a Gurobi model  

#============= Variables =============
p_DA = m.addVars(T, lb=0, ub=gb.GRB.INFINITY, name="p_DA") # day-ahead power bid

delta = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="delta") # decision variable for the power imbalance - can be negative
delta_up = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="delta_up") # surplus
delta_down = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="delta_down") # deficit

imbalance_revenue = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="I") # imbalance revenue - can be negative

y = m.addVars(2, T, OMEGA, vtype=gb.GRB.BINARY, name="y") # binary variables used to control the two-price logic

#============= Objective function =============
# Set objective function
obj = gb.quicksum(PI * (scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * p_DA[t] + imbalance_revenue[t,w]) for t in range(T) for w in range(OMEGA))
m.setObjective(obj, direction)

#============= Day-ahead power bid limits ============

#Upper limit is the nominal wind power
m.addConstrs(p_DA[t] <= WIND_CAPACITY for t in range(T))

#============= Power imbalance definitions ===============
m.addConstrs(delta[t,w] == scenarios[str(w)]['Wind Power [MW]'][t] - p_DA[t] for t in range(T) for w in range(OMEGA))
m.addConstrs(delta[t,w] == delta_up[t,w] - delta_down[t,w] for t in range(T) for w in range(OMEGA))

#============= Linearized conditional statements ===============
M = 1e6 # big-M constant

# if system is in a surplus and the imbalance is positive (NOT helping the system)
...

# if system is in a deficit and the imbalance is positive (IS helping the system)
...

# if system is in a surplus and the imbalance is negative (IS helping the system)
...

# if system is in a deficit and the imbalance is negative (NOT helping the system)
...



#Binary variable constraints
...

#============= Display and run model =============
m.update()
#m.display()
m.optimize()

In [ ]:
#============= Results =============
if m.status == gb.GRB.OPTIMAL:
    #initialization
    results = {}
    for scenario in range(OMEGA):
        df = pd.DataFrame(columns=['Hour', 'DA Price [EUR/MWh]', 'Wind Power [MW]', 'DA Bid [MW]', 'Imbalance [MW]', 'DA Profit [EUR]', 'Balancing Profit [EUR]', 'System State', 'Balancing Price Coefficient'])
        
        for t in range(T):
            df.loc[t] = [t, 
                         scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t], 
                         scenarios[str(scenario)]['Wind Power [MW]'][t], p_DA[t].x, 
                         delta[t,scenario].x, scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * p_DA[t].x, 
                         price_coeff[t,scenario].x * scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * delta[t,scenario].x, 
                         scenarios[str(scenario)]['System Balance State'][t], price_coeff[t,scenario].x]
        df['Total Profit [EUR]'] = df['DA Profit [EUR]'] + df['Balancing Profit [EUR]']

        df['Hour'] = df['Hour'].astype(int)
        df['System State'] = df['System State'].astype(int)
        df['System State'] = df['System State'].apply(lambda x: 'Deficit' if x == 1 else 'Surplus')
        df.set_index('Hour', inplace=True)
        results[scenario] = df.copy(deep=True)

    print('-----------------------------------------------')
    print('Objective value (expected profit): %.2f EUR' % m.objVal)
    print('-----------------------------------------------')
    print('Day-ahead bids:')
    average_hourly_profit = np.mean([results[w]['Total Profit [EUR]'] for w in range(OMEGA)], axis=0)

    for t in range(T):
        print('Hour %d | Bid: %.2f MW | Average Profit: %.2f EUR' % (t, p_DA[t].x, average_hourly_profit[t]))

    print('Sum of average profits: %.2f EUR' % np.sum(average_hourly_profit))
    print('-----------------------------------------------')
    print('Runtime: %f ms' % (m.Runtime * 1e3))
else:
    print("Optimization was not successful.")     